# Tarefa: Iris - Salvar e carregar o classificador

## Etapa 1: Importação das bibliotecas

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn, optim
from torch.nn import functional as F
torch.__version__

'1.13.1+cu116'

## Etapa 2: Base de dados

In [2]:
torch.manual_seed(123)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
base = pd.read_csv('/content/drive/MyDrive/Deep Learing de A à Z com PyTorch/Bases/iris.csv')
previsores = base.iloc[:, 0:4].values
classe = base.iloc[:, 4].values

In [5]:
encoder = LabelEncoder()
classe = encoder.fit_transform(classe)

In [6]:
previsores = torch.tensor(previsores, dtype = torch.float)
classe = torch.tensor(classe, dtype = torch.long)

In [7]:
dataset = torch.utils.data.TensorDataset(previsores, classe)
train_loader = torch.utils.data.DataLoader(dataset, batch_size = 10, shuffle = True)

## Etapa 3: Construção do modelo

In [8]:
classificador = nn.Sequential(
                nn.Linear(4, 8),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(8, 8),
                nn.ReLU(),
                nn.Dropout(0.1), 
                nn.Linear(8, 3),
                )

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classificador.parameters(), lr = 0.001, weight_decay = 0.0001)

## Etapa 4: Treinamento do modelo

In [10]:
for epoch in range(2000):
    
    running_loss = 0.
    running_accuracy = 0.
    
    for data in train_loader:
        inputs, labels = data

        optimizer.zero_grad()        

        outputs = classificador(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()

        running_loss += loss.item()

        ps = F.softmax(outputs)
        
        top_p, top_class = ps.topk(k = 1, dim = 1)
        
        equals = top_class == labels.view(*top_class.shape)
        
        running_accuracy += torch.mean(equals.type(torch.float))
     
    print('Época {:3d}: perda {:3.5f} - precisão {:3.5f}'.format(epoch + 1, running_loss/len(train_loader), running_accuracy/len(train_loader)))

<ipython-input-10-0ce894f40e7f>:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  ps = F.softmax(outputs)


Época   1: perda 1.16870 - precisão 0.28000
Época   2: perda 1.14063 - precisão 0.25333
Época   3: perda 1.12885 - precisão 0.16667
Época   4: perda 1.09960 - precisão 0.18667
Época   5: perda 1.11141 - precisão 0.29333
Época   6: perda 1.11828 - precisão 0.30667
Época   7: perda 1.10135 - precisão 0.31333
Época   8: perda 1.06157 - precisão 0.36667
Época   9: perda 1.06608 - precisão 0.26667
Época  10: perda 1.03932 - precisão 0.35333
Época  11: perda 1.03850 - precisão 0.30667
Época  12: perda 1.02611 - precisão 0.33333
Época  13: perda 1.01954 - precisão 0.35333
Época  14: perda 1.00724 - precisão 0.34667
Época  15: perda 0.99452 - precisão 0.34667
Época  16: perda 0.98376 - precisão 0.36000
Época  17: perda 0.98136 - precisão 0.43333
Época  18: perda 0.96018 - precisão 0.52000
Época  19: perda 0.94075 - precisão 0.60667
Época  20: perda 0.94860 - precisão 0.60667
Época  21: perda 0.91930 - precisão 0.65333
Época  22: perda 0.91931 - precisão 0.58000
Época  23: perda 0.91505 - preci

## Etapa 5: Salvar e carregar o classificador e classificar um registro

In [11]:
torch.save(classificador.state_dict(), 'checkpoint.pth')

In [12]:
state_dict = torch.load('checkpoint.pth')
classificador.load_state_dict(state_dict)

<All keys matched successfully>

In [13]:
novo = torch.tensor([[3.2, 4.5, 0.9, 1.1]], requires_grad = False)
classificador.eval()
previsao = classificador(novo)
previsao = F.softmax(previsao)
previsao = (previsao > 0.5).numpy()
if previsao[0][0] == True and previsao[0][1] == False and previsao[0][2] == False:
    print('Iris setosa')
elif previsao[0][0] == False and previsao[0][1] == True and previsao[0][2] == False:
    print('Iris virginica')
elif previsao[0][0] == False and previsao[0][1] == False and previsao[0][2] == True:
    print('Iris versicolor')

Iris setosa


<ipython-input-13-11b24fb0d888>:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  previsao = F.softmax(previsao)
